In [1]:
import numpy as np
import pandas as pd
import json
import nltk.data
from nltk.corpus import stopwords
from gensim.models import word2vec
import re

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import random as ra

## Some constants

In [2]:
title_size=20
label_size=18
legend_size=16
ticklabel_size=14
text_size=14

In [4]:
sns.set_palette("husl")
MachineLearning_color=(214/255.,95/255.,95/255.)
datascience_color=(72/255.,120/255.,207/255.)
compsci_color=(207/255., 120/255., 72/255.)

In [5]:
directory='/home/preston/Desktop/Programming/datasci/projects/blog_finder/data/reddit/'

## Load file and store all submissions, titles into a pandas df

In [ ]:
r_directory='/home/preston/Desktop/Programming/datasci/projects/blog_finder/data/reddit/'
se_directory='/home/preston/Desktop/Programming/datasci/projects/blog_finder/data/stackexchange/'
r_MachineLearning_filename='MachineLearning'
r_datascience_filename='datascience'
se_datascience_filename='datascience_questions'
r_statistics_filename='statistics'
r_compsci_filename='compsci'
se_stackoverflow_filename='stackoverflow_questions'
r_physics_filename='physics'
r_Cooking_filename='Cooking'
r_food_filename='food'

In [ ]:
r_MachineLearning_filehandle=open(r_directory+r_MachineLearning_filename)
r_datascience_filehandle=open(r_directory+r_datascience_filename)
se_datascience_filehandle=open(se_directory+se_datascience_filename)
r_statistics_filehandle=open(r_directory+r_statistics_filename)
r_compsci_filehandle=open(r_directory+r_compsci_filename)
se_stackoverflow_filehandle=open(se_directory+se_stackoverflow_filename)
r_physics_filehandle=open(r_directory+r_physics_filename)
r_Cooking_filehandle=open(r_directory+r_Cooking_filename)
r_food_filehandle=open(r_directory+r_food_filename)

In [ ]:
columns=['name', 'title']
r_MachineLearning_df=pd.DataFrame(columns=columns)
r_datascience_df=pd.DataFrame(columns=columns)
se_datascience_df=pd.DataFrame(columns=columns)
r_statistics_df=pd.DataFrame(columns=columns)
r_compsci_df=pd.DataFrame(columns=columns)
se_stackoverflow_df=pd.DataFrame(columns=columns)
r_physics_df=pd.DataFrame(columns=columns)
r_Cooking_df=pd.DataFrame(columns=columns)
r_food_df=pd.DataFrame(columns=columns)

In [ ]:
line=' '
while line:
    line=r_MachineLearning_filehandle.readline()
    line_json=json.loads(line)
    
    if line_json['name'].split('_')[0]=='t3':
        r_MachineLearning_df.loc[len(r_MachineLearning_df)]=[line_json['name'], line_json['title']]

In [ ]:
line=' '
while line:
    line = json.loads(r_datascience_filehandle.readline())
    if line['name'].split('_')[0]=='t3':
        r_datascience_df.loc[len(r_datascience_df)]=[line['name'], line['title']]

In [ ]:
line=' '
while line:
    line=se_datascience_filehandle.readline()
    line_json=json.loads(line)
    
    for item in line_json['items']:
        se_datascience_df.loc[len(se_datascience_df)]=[item['question_id'], item['title']]

In [ ]:
line=' '
while line:
    line=r_statistics_filehandle.readline()
    line_json=json.loads(line)
    
    if line_json['name'].split('_')[0]=='t3':
        r_statistics_df.loc[len(r_statistics_df)]=[line_json['name'], line_json['title']]

In [ ]:
line=' '
while line:
    line = json.loads(r_compsci_filehandle.readline())
    if line['name'].split('_')[0]=='t3':
        r_compsci_df.loc[len(r_compsci_df)]=[line['name'], line['title']]

In [ ]:
line=' '
i=0
while line:
    if i%100==0:
        print i
    line=se_stackoverflow_filehandle.readline()
    line_json=json.loads(line)
    
    for item in line_json['items']:
        se_stackoverflow_df.loc[len(se_stackoverflow_df)]=[item['question_id'], item['title']]
        
    i+=1

In [ ]:
line=' '
while line:
    line=r_physics_filehandle.readline()
    line_json=json.loads(line)
    
    if line_json['name'].split('_')[0]=='t3':
        r_physics_df.loc[len(r_physics_df)]=[line_json['name'], line_json['title']]

In [ ]:
line=' '
while line:
    line=r_Cooking_filehandle.readline()
    line_json=json.loads(line)
    
    if line_json['name'].split('_')[0]=='t3':
        r_Cooking_df.loc[len(r_Cooking_df)]=[line_json['name'], line_json['title']]

In [ ]:
line=' '
while line:
    line=r_food_filehandle.readline()
    line_json=json.loads(line)
    
    if line_json['name'].split('_')[0]=='t3':
        r_food_df.loc[len(r_food_df)]=[line_json['name'], line_json['title']]

## Lowercase all letters

In [ ]:
for df in [r_MachineLearning_df, r_datascience_df, se_datascience_df, r_statistics_df, r_compsci_df, se_stackoverflow_df,\
           r_physics_df, r_Cooking_df, r_food_df]:
    df['title']=df['title'].map(lambda x: x.lower())

## Remove all non-words
- 0-9
- !, @, #, etc.

In [ ]:
stop=stopwords.words('english')

In [ ]:
for df in [r_MachineLearning_df, r_datascience_df, se_datascience_df, r_statistics_df, r_compsci_df, se_stackoverflow_df,\
           r_physics_df, r_Cooking_df, r_food_df]:
    df['title']=df['title'].map(lambda x: ' '.join([word for word in x.split(' ') if word not in stop]))

## Create dictionary from submission titles

In [ ]:
r_MachineLearning_document=''
r_datascience_document=''
se_datascience_document=''
r_statistics_document=''
r_compsci_document=''
se_stackoverflow_document=''
r_physics_document=''
r_Cooking_document=''
r_food_document=''

for title in r_MachineLearning_df['title'].values:
    r_MachineLearning_document+=title+' '
    
for title in r_datascience_df['title'].values:
    r_datascience_document+=title+' '
    
for title in se_datascience_df['title'].values:
    se_datascience_document+=title+' '
    
for title in r_statistics_df['title'].values:
    r_statistics_document+=title+' '
    
for title in r_compsci_df['title'].values:
    r_compsci_document+=title+' '
    
for title in se_stackoverflow_df['title'].values:
    se_stackoverflow_document+=title+' '
    
for title in r_physics_df['title'].values:
    r_physics_document+=title+' '
    
for title in r_Cooking_df['title'].values:
    r_Cooking_document+=title+' '
    
for title in r_food_df['title'].values:
    r_food_document+=title+' '

In [ ]:
r_MachineLearning_word_list=filter(None,r_MachineLearning_document.split(' '))
r_datascience_word_list=filter(None,r_datascience_document.split(' '))
se_datascience_word_list=filter(None,se_datascience_document.split(' '))
r_statistics_word_list=filter(None,r_statistics_document.split(' '))
r_compsci_word_list=filter(None,r_compsci_document.split(' '))
se_stackoverflow_word_list=filter(None,se_stackoverflow_document.split(' '))
r_physics_word_list=filter(None,r_physics_document.split(' '))
r_Cooking_word_list=filter(None,r_Cooking_document.split(' '))
r_food_word_list=filter(None,r_food_document.split(' '))

In [ ]:
r_MachineLearning_word_dict={}
r_datascience_word_dict={}
se_datascience_word_dict={}
r_statistics_word_dict={}
r_compsci_word_dict={}
se_stackoverflow_word_dict={}
r_physics_word_dict={}
r_Cooking_word_dict={}
r_food_word_dict={}

################################################################################################################

for word in r_MachineLearning_word_list:
    if word in r_MachineLearning_word_dict:
        r_MachineLearning_word_dict[word]+=1
    else:
        r_MachineLearning_word_dict[word]=0
        
for key in r_MachineLearning_word_dict:
    r_MachineLearning_word_dict[key]=1.*r_MachineLearning_word_dict[key]/len(r_MachineLearning_word_list)

################################################################################################################    
    
for word in r_datascience_word_list:
    if word in r_datascience_word_dict:
        r_datascience_word_dict[word]+=1
    else:
        r_datascience_word_dict[word]=0
        
for key in r_datascience_word_dict:
    r_datascience_word_dict[key]=1.*r_datascience_word_dict[key]/len(r_datascience_word_list)
    
################################################################################################################
    
for word in se_datascience_word_list:
    if word in se_datascience_word_dict:
        se_datascience_word_dict[word]+=1
    else:
        se_datascience_word_dict[word]=0
        
for key in se_datascience_word_dict:
    se_datascience_word_dict[key]=1.*se_datascience_word_dict[key]/len(se_datascience_word_list)
    
################################################################################################################
    
for word in r_statistics_word_list:
    if word in r_statistics_word_dict:
        r_statistics_word_dict[word]+=1
    else:
        r_statistics_word_dict[word]=0
        
for key in r_statistics_word_dict:
    r_statistics_word_dict[key]=1.*r_statistics_word_dict[key]/len(r_statistics_word_list)
    
################################################################################################################
    
for word in r_compsci_word_list:
    if word in r_compsci_word_dict:
        r_compsci_word_dict[word]+=1
    else:
        r_compsci_word_dict[word]=0
        
for key in r_compsci_word_dict:
    r_compsci_word_dict[key]=1.*r_compsci_word_dict[key]/len(r_compsci_word_list)
        
################################################################################################################    
    
for word in se_stackoverflow_word_list:
    if word in se_stackoverflow_word_dict:
        se_stackoverflow_word_dict[word]+=1
    else:
        se_stackoverflow_word_dict[word]=0
        
for key in se_stackoverflow_word_dict:
    se_stackoverflow_word_dict[key]=1.*se_stackoverflow_word_dict[key]/len(se_stackoverflow_word_list)   
    
################################################################################################################
    
for word in r_physics_word_list:
    if word in r_physics_word_dict:
        r_physics_word_dict[word]+=1
    else:
        r_physics_word_dict[word]=0
        
for key in r_physics_word_dict:
    r_physics_word_dict[key]=1.*r_physics_word_dict[key]/len(r_physics_word_list)
    
################################################################################################################
    
for word in r_Cooking_word_list:
    if word in r_Cooking_word_dict:
        r_Cooking_word_dict[word]+=1
    else:
        r_Cooking_word_dict[word]=0
        
for key in r_Cooking_word_dict:
    r_Cooking_word_dict[key]=1.*r_Cooking_word_dict[key]/len(r_Cooking_word_list)
    
################################################################################################################
    
for word in r_food_word_list:
    if word in r_food_word_dict:
        r_food_word_dict[word]+=1
    else:
        r_food_word_dict[word]=0
        
for key in r_food_word_dict:
    r_food_word_dict[key]=1.*r_food_word_dict[key]/len(r_food_word_list)

## Get most common words, plot figure

In [ ]:
from copy import copy
def get_top_twenty_terms(dict):
    temp_dict=copy(dict)
    word_list=['' for i in range(20)]
    for i in range(20):
        word_list[i]=max(temp_dict, key=temp_dict.get)
        temp_dict={key: value for key, value in temp_dict.iteritems() if key!=word_list[i]}
    print word_list
    return word_list

In [ ]:
MachineLearning_most_common_words=get_top_twenty_terms(MachineLearning_word_dict)
datascience_most_common_words=get_top_twenty_terms(datascience_word_dict)

In [ ]:
from math import log

fig=plt.figure()

plt.scatter([i for i in range(len(MachineLearning_most_common_words))], [log(MachineLearning_word_dict[key]) for key in MachineLearning_most_common_words],\
           color=MachineLearning_color, label='/r/MachineLearning', s=24)
plt.scatter([i for i in range(len(datascience_most_common_words))], [log(datascience_word_dict[key]) for key in datascience_most_common_words],\
                                                                    color=datascience_color, label='/r/datascience', s=24)

plt.legend(loc='upper right', prop={'size':legend_size})

plt.xlim(-1,len(MachineLearning_most_common_words))
#plt.ylim(-.01,0.1)

fig.set_size_inches(12,10)
plt.title('most common terms in /r/MachineLearning and /r/datascience submissions', size = title_size)
plt.xlabel('term', size=label_size)
plt.ylabel('log normalized frequency', size=label_size)
plt.xticks([])
plt.yticks(size=ticklabel_size)


xoffset=0.0
yoffset=0.0

for i in range(len(MachineLearning_most_common_words)):
    xoffset1=xoffset
    yoffset1=yoffset
    xoffset2=-1*xoffset
    yoffset2=-1*yoffset
    ha1='right'
    va1='top'
    ha2='left'
    va2='bottom'
    if MachineLearning_word_dict[MachineLearning_most_common_words[i]] > datascience_word_dict[datascience_most_common_words[i]]:
        xoffset1=-1*xoffset
        yoffset1=-1*yoffset
        xoffset2=xoffset
        yoffset2=yoffset
        ha1='left'
        va1='bottom'
        ha2='right'
        va2='top'
    plt.text(i,\
             log(MachineLearning_word_dict[MachineLearning_most_common_words[i]]), \
             MachineLearning_most_common_words[i], rotation='45', color=MachineLearning_color, ha=ha1, va=va1, size=text_size)
    
    plt.text(i,\
             log(datascience_word_dict[datascience_most_common_words[i]])+0, \
             datascience_most_common_words[i], rotation='45', color=datascience_color, ha=ha2, va=va2, size=text_size)

plt.show()

## Get cosine distance between all subreddit dicts, plot matrix

In [ ]:
def cosine_distance(dict1, dict2):
    distance=0
    for key in dict1:
        if key in dict2:
            distance+=dict1[key]*dict2[key]
    return distance

In [ ]:
from math import log

In [ ]:
r_MachineLearning_r_datascience=log(cosine_distance(r_MachineLearning_word_dict, r_datascience_word_dict))
r_MachineLearning_se_datascience=log(cosine_distance(r_MachineLearning_word_dict, se_datascience_word_dict))
r_MachineLearning_r_statistics=log(cosine_distance(r_MachineLearning_word_dict, r_statistics_word_dict))
r_MachineLearning_r_compsci=log(cosine_distance(r_MachineLearning_word_dict, r_compsci_word_dict))
r_MachineLearning_se_stackoverflow=log(cosine_distance(r_MachineLearning_word_dict, se_stackoverflow_word_dict))
r_MachineLearning_r_physics=log(cosine_distance(r_MachineLearning_word_dict, r_physics_word_dict))
r_MachineLearning_r_Cooking=log(cosine_distance(r_MachineLearning_word_dict, r_Cooking_word_dict))
r_MachineLearning_r_food=log(cosine_distance(r_MachineLearning_word_dict, r_food_word_dict))

r_datascience_se_datascience=log(cosine_distance(r_datascience_word_dict, se_datascience_word_dict))
r_datascience_r_statistics=log(cosine_distance(r_datascience_word_dict, r_statistics_word_dict))
r_datascience_r_compsci=log(cosine_distance(r_datascience_word_dict, r_compsci_word_dict))
r_datascience_se_stackoverflow=log(cosine_distance(r_datascience_word_dict, se_stackoverflow_word_dict))
r_datascience_r_physics=log(cosine_distance(r_datascience_word_dict, r_physics_word_dict))
r_datascience_r_Cooking=log(cosine_distance(r_datascience_word_dict, r_Cooking_word_dict))
r_datascience_r_food=log(cosine_distance(r_datascience_word_dict, r_food_word_dict))

se_datascience_r_statistics=log(cosine_distance(se_datascience_word_dict, r_statistics_word_dict))
se_datascience_r_compsci=log(cosine_distance(se_datascience_word_dict, r_compsci_word_dict))
se_datascience_se_stackoverflow=log(cosine_distance(se_datascience_word_dict, se_stackoverflow_word_dict))
se_datascience_r_physics=log(cosine_distance(se_datascience_word_dict, r_physics_word_dict))
se_datascience_r_Cooking=log(cosine_distance(se_datascience_word_dict, r_Cooking_word_dict))
se_datascience_r_food=log(cosine_distance(se_datascience_word_dict, r_food_word_dict))

r_statistics_r_compsci=log(cosine_distance(r_statistics_word_dict, r_compsci_word_dict))
r_statistics_se_stackoverflow=log(cosine_distance(r_statistics_word_dict, se_stackoverflow_word_dict))
r_statistics_r_physics=log(cosine_distance(r_statistics_word_dict, r_physics_word_dict))
r_statistics_r_Cooking=log(cosine_distance(r_statistics_word_dict, r_Cooking_word_dict))
r_statistics_r_food=log(cosine_distance(r_statistics_word_dict, r_food_word_dict))

r_compsci_se_stackoverflow=log(cosine_distance(r_compsci_word_dict, se_stackoverflow_word_dict))
r_compsci_r_physics=log(cosine_distance(r_compsci_word_dict, r_physics_word_dict))
r_compsci_r_Cooking=log(cosine_distance(r_compsci_word_dict, r_Cooking_word_dict))
r_compsci_r_food=log(cosine_distance(r_compsci_word_dict, r_food_word_dict))

se_stackoverflow_r_physics=log(cosine_distance(se_stackoverflow_word_dict, r_physics_word_dict))
se_stackoverflow_r_Cooking=log(cosine_distance(se_stackoverflow_word_dict, r_Cooking_word_dict))
se_stackoverflow_r_food=log(cosine_distance(se_stackoverflow_word_dict, r_food_word_dict))

r_physics_r_Cooking=log(cosine_distance(r_physics_word_dict, r_Cooking_word_dict))
r_physics_r_food=log(cosine_distance(r_physics_word_dict, r_food_word_dict))

r_Cooking_r_food=log(cosine_distance(r_Cooking_word_dict, r_food_word_dict))

0. MachineLearning
1. datascience
2. statistics
3. compsci
4. physics
5. Cooking
6. food

In [ ]:
cosine_matrix=np.zeros((9,9))


cosine_matrix[0,1]=r_MachineLearning_r_datascience
cosine_matrix[1,0]=cosine_matrix[0,1]

cosine_matrix[0,2]=r_MachineLearning_se_datascience
cosine_matrix[2,0]=cosine_matrix[0,2]

cosine_matrix[0,3]=r_MachineLearning_r_statistics
cosine_matrix[3,0]=cosine_matrix[0,3]

cosine_matrix[0,4]=r_MachineLearning_r_compsci
cosine_matrix[4,0]=cosine_matrix[0,4]

cosine_matrix[0,5]=r_MachineLearning_se_stackoverflow
cosine_matrix[5,0]=cosine_matrix[0,5]

cosine_matrix[0,6]=r_MachineLearning_r_physics
cosine_matrix[6,0]=cosine_matrix[0,6]

cosine_matrix[0,7]=r_MachineLearning_r_Cooking
cosine_matrix[7,0]=cosine_matrix[0,7]

cosine_matrix[0,8]=r_MachineLearning_r_food
cosine_matrix[8,0]=cosine_matrix[0,8]



cosine_matrix[1,2]=r_datascience_se_datascience
cosine_matrix[2,1]=cosine_matrix[1,2]

cosine_matrix[1,3]=r_datascience_r_statistics
cosine_matrix[3,1]=cosine_matrix[1,3]

cosine_matrix[1,4]=r_datascience_r_compsci
cosine_matrix[4,1]=cosine_matrix[1,4]

cosine_matrix[1,5]=r_datascience_se_stackoverflow
cosine_matrix[5,1]=cosine_matrix[1,5]

cosine_matrix[1,6]=r_datascience_r_physics
cosine_matrix[6,1]=cosine_matrix[1,6]

cosine_matrix[1,7]=r_datascience_r_Cooking
cosine_matrix[7,1]=cosine_matrix[1,7]

cosine_matrix[1,8]=r_datascience_r_food
cosine_matrix[8,1]=cosine_matrix[1,8]




cosine_matrix[2,3]=se_datascience_r_statistics
cosine_matrix[3,2]=cosine_matrix[2,3]

cosine_matrix[2,4]=se_datascience_r_compsci
cosine_matrix[4,2]=cosine_matrix[2,4]

cosine_matrix[2,5]=se_datascience_se_stackoverflow
cosine_matrix[5,2]=cosine_matrix[2,5]

cosine_matrix[2,6]=se_datascience_r_physics
cosine_matrix[6,2]=cosine_matrix[2,6]

cosine_matrix[2,7]=se_datascience_r_Cooking
cosine_matrix[7,2]=cosine_matrix[2,7]

cosine_matrix[2,8]=se_datascience_r_food
cosine_matrix[8,2]=cosine_matrix[2,8]



cosine_matrix[3,4]=r_statistics_r_compsci
cosine_matrix[4,3]=cosine_matrix[3,4]

cosine_matrix[3,5]=r_statistics_se_stackoverflow
cosine_matrix[5,3]=cosine_matrix[3,5]

cosine_matrix[3,6]=r_statistics_r_physics
cosine_matrix[6,3]=cosine_matrix[3,6]

cosine_matrix[3,7]=r_statistics_r_Cooking
cosine_matrix[7,3]=cosine_matrix[3,7]

cosine_matrix[3,8]=r_statistics_r_food
cosine_matrix[8,3]=cosine_matrix[3,8]



cosine_matrix[4,5]=r_compsci_se_stackoverflow
cosine_matrix[5,4]=cosine_matrix[4,5]

cosine_matrix[4,6]=r_compsci_r_physics
cosine_matrix[6,4]=cosine_matrix[4,6]

cosine_matrix[4,7]=r_compsci_r_Cooking
cosine_matrix[7,4]=cosine_matrix[4,7]

cosine_matrix[4,8]=r_compsci_r_food
cosine_matrix[8,4]=cosine_matrix[4,8]



cosine_matrix[5,6]=se_stackoverflow_r_physics
cosine_matrix[6,5]=cosine_matrix[5,6]

cosine_matrix[5,7]=se_stackoverflow_r_Cooking
cosine_matrix[7,5]=cosine_matrix[5,7]

cosine_matrix[5,8]=se_stackoverflow_r_food
cosine_matrix[8,5]=cosine_matrix[5,8]



cosine_matrix[6,7]=r_physics_r_Cooking
cosine_matrix[7,6]=cosine_matrix[6,7]

cosine_matrix[6,8]=r_physics_r_food
cosine_matrix[8,6]=cosine_matrix[6,8]



cosine_matrix[7,8]=r_Cooking_r_food
cosine_matrix[8,7]=cosine_matrix[7,8]

In [ ]:
import matplotlib.cm as cmap
fig=plt.figure()
cmap=cmap.gray
im=plt.imshow(cosine_matrix, interpolation='none', vmin=-8.5, vmax=-5.5, cmap=cmap)

plt.xlim(-0.5,8.5)
plt.ylim(-0.5,8.5)

plt.title('submissions log cosine similarity', size=title_size)
for i in range(0, 9):
    plt.plot([i+.5,i+.5],[-10,10], color=(1,1,1))
    plt.plot([-10,10],[i+.5,i+.5], color=(1,1,1))
    
plt.gca().xaxis.grid(False)
plt.gca().yaxis.grid(False)

plt.xticks([0,1,2, 3, 4, 5, 6, 7, 8],\
           ['/r/Machine\nLearning', '/r/datascience', 'stackexchange.\ndatascience', '/r/statistics',\
            '/r/compsci', 'stackoverflow', '/r/physics', '/r/Cooking', '/r/food'], rotation='15', font_properties='bold')
plt.yticks([0,1,2, 3, 4, 5, 6, 7, 8],\
           ['/r/Machine\nLearning', '/r/datascience', 'stackexchange.\ndatascience', '/r/statistics',\
            '/r/compsci', 'stackoverflow', '/r/physics', '/r/Cooking', '/r/food'], font_properties='bold')

for i in range(0,9):
    for j in range(i,9):
        plt.text(j-.5+.1, i-.5+.1, str(round(cosine_matrix[i,j],3)), color=(1.,0,0), ha='left', va='bottom', size=14,\
                font_properties='bold')

plt.colorbar(im, fraction=0.046, pad=0.04)
fig.set_size_inches(10,10)


plt.show()

## Cosine distance between search and subreddit dict for different search terms and subreddits

In [ ]:
def cosine_distance(search, dict1):
    distance=0
    num_words=len(search.split(' '))
    for word in search.split(' '):
        if word in dict1:
            distance+=1.*dict1[word]/num_words
    return distance

In [ ]:
searches=['machine learning', 'data science', 'big data', 'python r', 'regression', 'classification', 'neural network']
r_MachineLearning_distance={term: 0 for term in searches}
r_datascience_distance={term: 0 for term in searches}
se_datascience_distance={term: 0 for term in searches}
for search in searches:
    r_MachineLearning_distance[search]=cosine_distance(search, r_MachineLearning_word_dict)
    r_datascience_distance[search]=cosine_distance(search, r_datascience_word_dict)
    se_datascience_distance[search]=cosine_distance(search, se_datascience_word_dict)

In [ ]:
fig = plt.figure()

plt.bar([4*i for i in range(len(searches))], [r_MachineLearning_distance[searches[i]] for i in range(len(searches))],\
        alpha=0.75, color = MachineLearning_color, label='/r/MachineLearning')
plt.bar([4*i+2*ra.random()-1. for i in range(len(searches))], [r_datascience_distance[searches[i]] for i in range(len(searches))],\
        alpha=0.75, color = datascience_color, label='/r/datascience')
plt.bar([4*i+2*ra.random()-1. for i in range(len(searches))], [se_datascience_distance[searches[i]] for i in range(len(searches))],\
        alpha=0.75, color = (106/255., 204/255., 101/255.), label='datascience.stackexchange')

plt.legend(prop={'size': legend_size})
plt.xlim(-2, (4*len(searches)-1))
    
fig.set_size_inches(12,10)
plt.title('cosine similarity of search phrase and community submissions vector', size=title_size)
plt.xlabel('search terms', size=label_size)
plt.ylabel('normalized cosine distance', size=label_size)

plt.xticks([4*i+0.5 for i in range(len(searches))], ['\''+'\n'.join(search.split( ))+'\'' for search in searches], size = ticklabel_size)
plt.yticks(size=ticklabel_size)
plt.show()

In [ ]:
blog_list=[]
for link in MachineLearning_df['link'].values:
    if 'blog' in link:
        blog_list.append(link)

print len(blog_list)
        
for blog in blog_list:
    print blog

#sort links by popularity (upvotes with respect to  